In [1]:
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [2]:
with open('key/sk_open_api_key.txt') as f_:
    sk_key = f_.read()

In [3]:
len(sk_key)

40

In [71]:
df_read = pd.read_csv('stn_code.csv')
stn = '수원역'  # 입력할 역
stn_code = df_read[df_read['stationName'] == stn]['stationCode'].values[0]
stn_code

'P155'

In [72]:
# 혼잡도
station_code = stn_code
dow = 'MON' # 대문자
hh = '08'
url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow={dow}&hh={hh}"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()


In [73]:
response.keys()

dict_keys(['status', 'contents'])

In [74]:
response['contents'].keys()

dict_keys(['subwayLine', 'stationName', 'stationCode', 'stat', 'statStartDate', 'statEndDate'])

In [75]:
len(response['contents']['stat'])

34

In [76]:
response['contents']['stat'][0]['startStationName']

'의정부역'

In [77]:
response['contents']['stationName']

'수원역'

In [89]:
response['contents']['stat'][15]['data']

[{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '30', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '50', 'congestionTrain': 0}]

In [79]:
temp_list = []
temp_list2 = []
for stat in response['contents']['stat']:
    if stat['updnLine'] == 1:
        temp_list.append(pd.DataFrame(stat['data']))
    elif stat['updnLine'] == 0:
        temp_list2.append(pd.DataFrame(stat['data']))
# temp_list = temp_list[0]
# temp_list
# temp_list2 = temp_list2[0]
# temp_list2


In [63]:
data1 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 1]
data2 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 0]

In [80]:
df = pd.concat(temp_list)
df['updn'] = 1
df_ = pd.concat(temp_list2)
df_['updn'] = 0
df = pd.concat((df,df_))
df

,dow,hh,mm,congestionTrain,updn
0,MON,08,00,0,1
1,MON,08,10,0,1
2,MON,08,20,0,1
3,MON,08,30,8,1
4,MON,08,40,8,1
...,...,...,...,...,...
1,MON,08,10,0,0
2,MON,08,20,0,0
3,MON,08,30,0,0
4,MON,08,40,0,0


In [108]:
# 최대혼잡도
df_res = df[df['congestionTrain'] != 0].copy().reset_index(drop=True)
df_res.drop(columns=['dow','hh','mm'],inplace=True)


In [107]:
df_res = df_res.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,15
1,1,32


In [109]:
df_res = df_res.groupby('updn')['congestionTrain'].agg('min').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,4
1,1,6


In [ ]:
df = pd.concat(data1)
df['updn'] = 1
df_ = pd.concat(data2)
df_['updn'] = 0
df = pd.concat((df,df_))
df

In [29]:
mm = 10

In [30]:
df_res = df[df['congestionTrain'] != 0].copy()
df_res = df_res[df_res['mm'] == mm].copy()
df_res.drop(columns=['dow','hh','mm'],inplace=True)
df_res = df.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,201
1,1,72


In [31]:
#######################
import requests

url = "https://apis.openapi.sk.com/puzzle/subway/meta/stations?type=train"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()

In [32]:
response.keys()

dict_keys(['status', 'contents'])

In [34]:
re = response['contents']

In [35]:
df = pd.DataFrame(re)

In [36]:
df

,subwayLine,stationName,stationCode
0,1호선,소요산역,100
1,1호선,동두천역,101
2,1호선,보산역,102
3,1호선,동두천중앙역,103
4,1호선,지행역,104
...,...,...,...
477,5호선,방이역,P551
478,5호선,오금역,P552
479,5호선,개롱역,P553
480,5호선,거여역,P554


In [37]:
df.to_csv('stn_code.csv',index=False)

In [50]:
stn = '소요산역'
selected_code = df[df['stationName'] == stn]['stationCode'].values[0]
selected_code

'100'

In [41]:
df.loc[df['stationName'] == stn, 'stationCode']

Series([], Name: stationCode, dtype: object)

In [ ]:
def get_stn_code(stn):
    